In [1]:
from qiskit import *
import numpy as np
from colorama import Fore, Back, Style 
from qiskit.aqua.components.optimizers import COBYLA
import random as rand

In [2]:
def coeff(N1,N2):
	return (np.dot(N1.conjugate().transpose(),N2)).trace()*0.25

In [3]:
def prepare_ansatz(parameter):
    
    qr=QuantumRegister(2)
    cr=ClassicalRegister(2)
    circuit=QuantumCircuit(qr,cr)
    
    #Preparing general parameterized ansatz for any two-qubit Hamiltonian
    
    circuit.u3(parameter[0],parameter[1],parameter[2],qr[0])
    circuit.u3(parameter[3],parameter[4],parameter[5],qr[1])

    circuit.cx(qr[1],qr[0])

    circuit.u3(parameter[6],parameter[7],parameter[8],qr[0])
    circuit.u3(parameter[9],parameter[10],parameter[11],qr[1])

    circuit.cx(qr[0],qr[1])

    circuit.u3(parameter[12],parameter[13],parameter[14],qr[0])
    circuit.u3(parameter[15],parameter[16],parameter[17],qr[1])

    circuit.cx(qr[1],qr[0])

    circuit.u3(parameter[18],parameter[19],parameter[20],qr[0])
    circuit.u3(parameter[21],parameter[22],parameter[23],qr[1])

    return circuit


In [4]:
def vqe_circuit(term, parameter):
	circuit=prepare_ansatz(parameter)
	qr=circuit.qregs[0]
	cr=circuit.cregs[0]
	if term=='XX':
		circuit.h(qr[0])
		circuit.h(qr[1])		
	elif term=='YY':
		circuit.u2(0,np.pi/2,qr[0])
		circuit.u2(0,np.pi/2,qr[1])
	elif term=='IX':
		circuit.h(qr[1])		
	elif term=='IY':
		circuit.u2(0,np.pi/2,qr[1])	
	elif term== 'XI':
		circuit.h(qr[0])	
	elif term=='YI':
		circuit.u2(0,np.pi/2,qr[0])
	elif term=='XY':
		circuit.h(qr[0])
		circuit.u2(0,np.pi/2,qr[1])
	elif term=='XZ':
		circuit.h(qr[0])
	elif term=='YX':
		circuit.u2(0,np.pi/2,qr[0])
		circuit.h(qr[1])
	elif term=='YZ':
		circuit.u2(0,np.pi/2,qr[0])
	elif term=='YY':
		circuit.u2(0,np.pi/2,qr[0])
		circuit.u2(0,np.pi/2,qr[1])
	elif term=='ZZ' or 'IZ' or 'ZI'or 'II':
		print(" ")
	else:
		raise ValueError("Wrong input for Pauli decomposition")
	circuit.measure(qr,cr)
	return circuit #returns final variational form of quantum circuit for a Pauli term

In [5]:
def term_expectation(term, circuit):
    shots=10000
    simulator=Aer.get_backend('qasm_simulator')
    result=execute(circuit,backend=simulator,shots=10000).result()
    counts=result.get_counts()
    if term in ['IX','IY','IZ']:
        bitstring_eigenvalue={'00':+1,'01':-1,'10':+1,'11':-1}
    elif term in ['XI','YI','ZI']:
        bitstring_eigenvalue={'00':+1,'01':+1,'10':-1,'11':-1}
    elif term in ['II']:
        bitstring_eigenvalue={'00':+1,'01':+1,'10':+1,'11':+1}
    else:
        bitstring_eigenvalue={'00':+1,'01':-1,'10':-1,'11':+1}
    expectation=0
    for bitstring in counts:
        expectation+=(bitstring_eigenvalue[bitstring]*counts[bitstring])/shots
    return expectation #returns expectation for a Pauli term


In [6]:
def objective_function(parameter):
    global counter
    counter+=1
    new_energy=0.0
    for term in pauli_dict:
        circuit=vqe_circuit(term,parameter)
        component=pauli_dict[term]*term_expectation(term,circuit)
        new_energy+=component 
    print(Fore.GREEN)
    print("Iteration ",counter," :",new_energy)
    print(Style.RESET_ALL)
    return new_energy

In [7]:
H=np.array([[1,0,0,0],[0,0,-1,0],[0,-1,0,0],[0,0,0,1]])
I= np.array([[1,0],[0,1]])
X=np.array([[0,1],[1,0]])
Y=np.array([[0,complex(0,-1)],[complex(0,1),0]])
Z=np.array([[1,0],[0,-1]])
matrix_label=['I','X','Y','Z']
matrix_list=[I,X,Y,Z]
pauli_dict={}
counter=0
print ("The Pauli decomposition of the given Hamiltonian \n\n",H, "is:\n")
for i in range(4):
    for j in range(4):
        matrix_name=matrix_label[i]+matrix_label[j]
        tensor=np.kron(matrix_list[i],matrix_list[j])
        coefficient=coeff(tensor,H)
        if coefficient!=0.0:
            pauli_dict[matrix_name]=coefficient	
            print(matrix_name,"  ",coefficient)
            

#Intializing COBYLA with maximum iteration 500 and tolerance 1e-4
optimizer = COBYLA(maxiter=500, tol=0.0001)

#Running COBYLA optimizer
parameter = np.random.rand(24)
output = optimizer.optimize(num_vars=24, objective_function=objective_function, initial_point=parameter)

#Calculating the lowest energy with the final parameters output[0]
lowest_energy=objective_function(output[0])


print("The lowest energy from VQE is", lowest_energy)
print("Parameters Found:", output[0])
            
            
            
            
            
            





The Pauli decomposition of the given Hamiltonian 

 [[ 1.   0.   0.   0. ]
 [ 0.  -1.5  0.   0. ]
 [ 0.   0.  -1.   0. ]
 [ 0.   0.   0.   1. ]] is:

II    -0.125
IZ    0.125
ZI    -0.125
ZZ    1.125
 
 
 
 

Iteration  1  : 0.5955750000000001

 
 
 
 

Iteration  2  : -0.02099999999999997

 
 
 
 

Iteration  3  : -0.15545000000000006

 
 
 
 

Iteration  4  : -0.16104999999999994

 
 
 
 

Iteration  5  : -0.36245

 
 
 
 

Iteration  6  : -0.35387500000000005

 
 
 
 

Iteration  7  : -0.38530000000000003

 
 
 
 

Iteration  8  : -0.667825

 
 
 
 

Iteration  9  : -0.9546000000000001

 
 
 
 

Iteration  10  : -0.620625

 
 
 
 

Iteration  11  : -0.8956

 
 
 
 

Iteration  12  : -0.97475

 
 
 
 

Iteration  13  : -0.51515

 
 
 
 

Iteration  14  : -1.3413750000000002

 
 
 
 

Iteration  15  : -1.3045750000000003

 
 
 
 

Iteration  16  : -0.6630999999999999

 
 
 
 

Iteration  17  : -1.106325

 
 
 
 

Iteration  18  : -1.39385

 
 
 
 

Iteration  19  : -1.39385

 
 
 
 



 
 
 

Iteration  163  : -1.4975749999999999

 
 
 
 

Iteration  164  : -1.497175

 
 
 
 

Iteration  165  : -1.497875

 
 
 
 

Iteration  166  : -1.49775

 
 
 
 

Iteration  167  : -1.4983250000000001

 
 
 
 

Iteration  168  : -1.497925

 
 
 
 

Iteration  169  : -1.498075

 
 
 
 

Iteration  170  : -1.4973

 
 
 
 

Iteration  171  : -1.497525

 
 
 
 

Iteration  172  : -1.497725

 
 
 
 

Iteration  173  : -1.4972249999999998

 
 
 
 

Iteration  174  : -1.49645

 
 
 
 

Iteration  175  : -1.4972750000000001

 
 
 
 

Iteration  176  : -1.4979749999999998

 
 
 
 

Iteration  177  : -1.4975749999999999

 
 
 
 

Iteration  178  : -1.4966499999999998

 
 
 
 

Iteration  179  : -1.4980749999999998

 
 
 
 

Iteration  180  : -1.49785

 
 
 
 

Iteration  181  : -1.497525

 
 
 
 

Iteration  182  : -1.4980250000000002

 
 
 
 

Iteration  183  : -1.4976

 
 
 
 

Iteration  184  : -1.4964749999999998

 
 
 
 

Iteration  185  : -1.4977

 
 
 
 

Iteration  186  : -1.4977999